In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data, wb
import datetime

import plotly.graph_objects as go

In [2]:
start = pd.to_datetime('2012-02-04')
end = pd.to_datetime('2017-05-29')

In [3]:
tesla = data.DataReader('TSLA', 'yahoo', start, end)

In [4]:
tesla.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2012-02-06,6.380,6.210,6.220,6.360,3260500,6.360
2012-02-07,6.360,6.164,6.360,6.320,5108000,6.320
2012-02-08,6.402,6.258,6.320,6.386,3118500,6.386
2012-02-09,6.580,6.286,6.400,6.516,6385500,6.516
2012-02-10,6.454,5.968,6.452,6.220,9371000,6.220


## ICHIMOKU APPLICATION 

In [10]:
#tenkan Sen (Conversion Line) = (9 period High + 9 period Low)/2
nine_period_high = tesla['High'].rolling(window=9).max()
nine_period_low= tesla['Low'].rolling(window=9).min()
tesla["tenkan_sen"] = (nine_period_high + nine_period_low)/2

#Kinjun Sen (Base Line) = (26 period High + 26 period Low)/2
period26_high = tesla['High'].rolling(window=26).max()
period26_low= tesla['Low'].rolling(window=26).min()
tesla["kinjun_sen"] = (period26_high + period26_low)/2

#Senkou Span A (Leading Span A) = (Conversion Line + Base Line)/2 ------- 26 sessions backwards
tesla["senkou_span_a"] = ((tesla["tenkan_sen"] + tesla["kinjun_sen"])/2).shift(26)

#Senkou Span B (Leading Span B) = (52 period High + 52 period Low)/2 ------- 26 sessions backwards
period52_high = tesla['High'].rolling(window=52).max()
period52_low= tesla['Low'].rolling(window=52).min()
tesla["senkou_span_b"] = ((period52_high + period52_low)/2).shift(26)

# The most current closing price plotted 26 time periods behind (optional) ---- chikou span
tesla['chikou_span'] = tesla['Close'].shift(-26)

In [11]:
tesla

,High,Low,Open,Close,Volume,Adj Close,tenkan_sen,kinjun_sen,senkou_span_a,senkou_span_b,chikou_span
Date,,,,,,,,,,,
2012-02-06,6.380000,6.210000,6.220000,6.360000,3260500,6.360000,NaN,NaN,NaN,NaN,7.058
2012-02-07,6.360000,6.164000,6.360000,6.320000,5108000,6.320000,NaN,NaN,NaN,NaN,7.000
2012-02-08,6.402000,6.258000,6.320000,6.386000,3118500,6.386000,NaN,NaN,NaN,NaN,7.064
2012-02-09,6.580000,6.286000,6.400000,6.516000,6385500,6.516000,NaN,NaN,NaN,NaN,6.996
2012-02-10,6.454000,5.968000,6.452000,6.220000,9371000,6.220000,NaN,NaN,NaN,NaN,6.992
...,...,...,...,...,...,...,...,...,...,...,...
2017-05-22,62.874001,61.360001,62.560001,62.070000,21646000,62.070000,63.231001,61.841999,57.740999,55.573999,NaN
2017-05-23,62.146000,60.695999,62.091999,60.771999,21592000,60.771999,63.048000,61.841999,58.216999,55.573999,NaN
2017-05-24,62.200001,61.080002,61.301998,62.043999,25166500,62.043999,63.048000,61.841999,58.216999,55.573999,NaN


In [23]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=tesla.index, y=tesla['chikou_span'], line=dict(color='red', width=.8), name='Chikou_span'))
fig.add_trace(go.Scatter(x=tesla.index, y=tesla['tenkan_sen'], line=dict(color='royalblue', width=.8), name='tenkan_sen'))
fig.add_trace(go.Scatter(x=tesla.index, y=tesla['kinjun_sen'], line=dict(color='orange', width=.8), name='kinjun_sen'))
fig.add_trace(go.Scatter(x=tesla.index, y=tesla['senkou_span_a'], line=dict(color='black', width=.8), name='senkou_span_a'))
fig.add_trace(go.Scatter(x=tesla.index, y=tesla['senkou_span_b'], line=dict(color='purple', width=1.8), name='senkou_span_b'))

fig.add_trace(go.Candlestick(x=tesla.index, open = tesla['Open'], close = tesla['Close'], high = tesla['High'], low = tesla['Low'], name='Market Data'))

fig.update_layout(
    title = 'Ichimoku Strategy',
    yaxis_title = 'Tesla Stock Price (USD/share)')

fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1 , label='1m', step='month' , stepmode='backward'),
            dict(count=6 , label='6m', step='month' , stepmode='backward'),
            dict(count=1 , label='YTD', step='year' , stepmode='todate'),
            dict(count=1 , label='1', step='year' , stepmode='backward'),
            dict(step='all')
        ])
    )
)


fig.show()